In [1]:
import PyPDF2
import numpy as np
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:


def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text


def generate_question_and_answer(model, tokenizer, seed_text, max_sequence_len, num_words_to_generate):
    for _ in range(num_words_to_generate):
        # Tokenize the seed text
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        # Truncate sequences to a fixed length
        encoded = pad_sequences(
            [encoded], maxlen=max_sequence_len, truncating='pre')
        # Predict the next word
        y_pred = model.predict_classes(encoded, verbose=0)
        # Map predicted word index to word
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == y_pred:
                predicted_word = word
                break
        # Add predicted word to the seed text for the next prediction
        seed_text += " " + predicted_word
    return seed_text



In [3]:

# Paths to the three PDF files
pdf1_path = "file1.pdf"
pdf2_path = "file2.pdf"
pdf3_path = "file3.pdf"

# Extract text from PDF files
pdf1_text = extract_text_from_pdf(pdf1_path)
pdf2_text = extract_text_from_pdf(pdf2_path)
pdf3_text = extract_text_from_pdf(pdf3_path)

# Combine texts
combined_text = pdf1_text + pdf2_text + pdf3_text


In [4]:
combined_text

"Prestazioni occasionali: le novità 2023 della Legge di Bilancio \nCon la Circolare n. 6/2023 l’Inps illustra le nuove disposizioni introdo Ʃe dall’art. 1, c. 342 e 343, della L. \n29 dicembre 2022, n. 197 \nDi Laura Biarella  Avvocato in Perugia  Pubblicato il 01/02/2023    Con una Circolare 19 gennaio 2023, n. 6 \n(testo in calce) l’Inps ha fornito chiarimen Ɵ in materia di prestazioni occasionali, con riferimento alle \nnuove disposizioni introdo Ʃe dall’art. 1 , c. 342 e 343, della Legge di Bilancio per il 2023.    Sommario        \nLa norma Ɵva      Limi Ɵ economici per l’accesso al Libre Ʃo Famiglia e al Contra Ʃo di prestazione \noccasionale      Nuovo limite dimensionale degli u Ɵlizzatori del Contra Ʃo di prest azione occasionale      \nDivieto di u Ɵlizzo del Contra Ʃo di prestazione occasionale per l’agricoltura    La norma Ɵva    L’art. 54 -bis \ndel d.l. n. 50/2017, introdo Ʃo, in sede di conversione, dalla L. n. 96/2017, ha fornito una compiuta \ndisciplina delle p restaz

In [5]:

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([combined_text])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences([combined_text])[0]


In [6]:

# Generate input sequences and labels
input_sequences = []
max_sequence_len = 50
for i in range(0, len(sequences) - max_sequence_len, 1):
    input_seq = sequences[i:i + max_sequence_len]
    input_sequences.append(input_seq)

# Convert input sequences to numpy arrays
input_sequences = np.array(input_sequences)

# Split input sequences into X and y
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Build the model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_sequence_len - 1),
    Bidirectional(LSTM(150, return_sequences=True)),
    Bidirectional(LSTM(150)),
    Dense(vocab_size, activation='softmax')
])


In [7]:

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


In [8]:

# Train the model
model.fit(X, y, batch_size=128, epochs=100)


Epoch 1/100


125/125 [==============================] - 38s 255ms/step - loss: 6.7513 - accuracy: 0.0512
Epoch 2/100
125/125 [==============================] - 33s 263ms/step - loss: 6.3453 - accuracy: 0.0541
Epoch 3/100
125/125 [==============================] - 31s 247ms/step - loss: 6.1236 - accuracy: 0.0653
Epoch 4/100
125/125 [==============================] - 34s 269ms/step - loss: 5.9293 - accuracy: 0.0752
Epoch 5/100
125/125 [==============================] - 32s 252ms/step - loss: 5.7415 - accuracy: 0.0902
Epoch 6/100
125/125 [==============================] - 31s 251ms/step - loss: 5.5588 - accuracy: 0.1005
Epoch 7/100
125/125 [==============================] - 31s 247ms/step - loss: 5.3862 - accuracy: 0.1123
Epoch 8/100
125/125 [==============================] - 31s 250ms/step - loss: 5.2190 - accuracy: 0.1341
Epoch 9/100
125/125 [==============================] - 31s 247ms/step - loss: 5.0813 - accuracy: 0.1485
Epoch 10/100
125/125 [==============================] - 31s 24

In [14]:

# Save the trained model
save_model(model, 'pdf_rnn_model.keras')


In [10]:

# Load the trained model
model = load_model('pdf_rnn_model.keras')

In [11]:
def generate_question_and_answer(model, tokenizer, seed_text, max_sequence_len, num_words_to_generate):
    for _ in range(num_words_to_generate):
        # Tokenize the seed text
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        # Truncate sequences to a fixed length
        encoded = pad_sequences(
            [encoded], maxlen=max_sequence_len - 1)#, truncating='pre')
        # Predict the next word probabilities
        y_pred_probs = model.predict(encoded, verbose=0)[0]
        # Get the index of the word with the highest probability
        predicted_word_index = np.argmax(y_pred_probs)
        # Map predicted word index to word
        predicted_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                predicted_word = word
                break
        # Add predicted word to the seed text for the next prediction
        seed_text += " " + predicted_word
    # Convert generated text to ASCII
    return seed_text.encode('ascii', 'ignore').decode('ascii')

In [16]:

# Generate a question prompt
seed_text = input("Enter a seed text to generate a question: ")
num_words_to_generate = int(
    input("Enter the number of words to generate in the question: "))

# Generate question and answer using the trained model
generated_question = generate_question_and_answer(
    model, tokenizer, seed_text, max_sequence_len, num_words_to_generate)

# Display the generated question
print("Generated Question:", generated_question)

Generated Question: Il Decreto Legge 24 aprile 2017 are il regime non possono a vit lavora ve sporadiche


In [13]:
save_model(model, 'pdf_rnn_model.h5')

# Load the trained model
model = load_model('pdf_rnn_model.h5')

# Generate a question prompt
seed_text = input("Enter a seed text to generate a question: ")
num_words_to_generate = int(
    input("Enter the number of words to generate in the question: "))

# Generate question and answer using the trained model
generated_question = generate_question_and_answer(
    model, tokenizer, seed_text, max_sequence_len, num_words_to_generate)

# Display the generated question
print("Generated Question:", generated_question)

C:\Users\stefa\AppData\Local\Temp\ipykernel_15980\2647120204.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, 'pdf_rnn_model.h5')


Generated Question: Di Laura Biarella  Avvocato in Perugia il reddito non possono essere disoccupa  lavit esercitata non superiore a lavoro autonomo occasionale non possono u lizzatore a partenza e non non non non non non non non non non non la vit professionale sul luogo la prestazione occasionale  tenuto ad ritenute della contribuente per la colonna
